In [1]:
using SHA
using Symbolics
using LinearAlgebra

In [2]:
message = "Hello, world!"
hash = sha2_512(message)

64-element Vector{UInt8}:
 0xc1
 0x52
 0x7c
 0xd8
 0x93
 0xc1
 0x24
 0x77
 0x3d
 0x81
 0x19
 0x11
 0x97
    ⋮
 0x9a
 0x0b
 0xb2
 0x69
 0xf5
 0x1f
 0x6e
 0x7a
 0x57
 0xf7
 0x94
 0x21

In [3]:
bitstring512=""
for i in hash
    bitstring512=bitstring512*bitstring(i)
end
bitstring512

"11000001010100100111110011011000100100111100000100100100011101110011110110000001000110010001000110010111000011001000111111100110111010000101011111010110110111110101110111001001001000100110101111011000101000010110000001100001010011000000110011011001011000111010010011011101111010100010101110010100101110110111110100110110000000100001111011111001110110000110010111010101110011101010001010010100101010000010110111010100100110100000101110110010011010011111010100011111011011100111101001010111111101111001010000100001"

# Bit string of length 512 

In [60]:
# A512=[]
# for i in 1:512
#     b=Int(round(rand()))
#     append!(A512,b)
# end

In [4]:
A512=[]
for i in bitstring512
#     print(Int(i)-48)
    append!(A512,Int(i)-48)
end
# A512

In [15]:
# coefficients
coeffs=[]
last12=A512[501:512,]
for i in 0:3
    b3=last12[(1+3*i):3*(i+1),]
    coef=0
    for j in 1:3
        coef=coef+2^(3-j)*b3[j]
    end
    coef=mod(coef,6)
    append!(coeffs,coef)
end
println(coeffs)

Any[2, 0, 4, 1]


In [7]:
#first 300, mapping from 50 variables
first300=A512[1:300]
vars=[]
for i in 1:50
    v=first300[(1+6*(i-1)):(6*i),]
    vn=0
    for j in 1:6
        vn=vn+2^(6-j)*v[j]
    end
    vn=vn+1
    append!(vars,vn)
end
println(vars)

Any[49, 22, 10, 61, 55, 10, 16, 2, 10, 8, 29, 62, 33, 18, 37, 18, 38, 49, 51, 16, 58, 47, 34, 24, 54, 46, 62, 30, 51, 19, 10, 44, 55, 11, 6, 33, 25, 21, 49, 13, 55, 23, 15, 37, 56, 31, 41, 44, 38, 12]


In [12]:
@variables x[1:64]
middle200=A512[301:500]
mons=[]
for i in 1:5
    b40=middle200[(1+40*(i-1)):(40*i)]

    for j in 1:4
        b10=b40[(1+10*(j-1)):(10*j)]
        mon=1

        for k in 1:10
            if b10[k]==1 

                if vars[10*(j-1)+k]!=0
                    mon=mon*x[vars[10*(j-1)+k]]
                end
            end
        end
        append!(mons,mon)
        
    end
end
mons

20-element Vector{Any}:
          (x[10]^3)*x[2]*x[8]*x[16]*x[49]*x[61]
                  x[18]*x[37]*x[49]*x[51]*x[62]
                                          x[62]
 x[6]*x[11]*x[13]*x[21]*x[25]*x[44]*x[49]*x[55]
               (x[10]^2)*x[2]*x[49]*x[55]*x[61]
                        x[16]*x[18]*x[37]*x[49]
      x[19]*x[24]*x[30]*x[46]*x[47]*x[51]*x[58]
                   x[6]*x[11]*x[25]*x[49]*x[55]
                          x[2]*x[8]*x[10]*x[55]
                              x[33]*x[37]*x[38]
            x[19]*x[30]*x[34]*x[46]*x[51]*x[54]
                        x[11]*x[13]*x[25]*x[44]
                                (x[10]^2)*x[49]
            x[18]*x[29]*x[33]*x[37]*x[51]*x[62]
            x[19]*x[30]*x[34]*x[47]*x[51]*x[54]
                  x[10]*x[11]*x[13]*x[33]*x[44]
         (x[10]^3)*x[8]*x[16]*x[22]*x[49]*x[61]
              (x[18]^2)*x[29]*x[37]*x[38]*x[51]
      x[19]*x[24]*x[30]*x[46]*x[47]*x[51]*x[62]
       x[6]*x[10]*x[11]*x[13]*x[25]*x[33]*x[44]

In [17]:
polys=[]
for i in 1:5
    pol=0
    for j in 1:4
        pol=pol+mons[4*(i-1)+j]*coeffs[j]
    end
    append!(polys,pol)
end
polys

5-element Vector{Any}:
                                4x[62] + 2(x[10]^3)*x[2]*x[8]*x[16]*x[49]*x[61] + x[6]*x[11]*x[13]*x[21]*x[25]*x[44]*x[49]*x[55]
                   x[6]*x[11]*x[25]*x[49]*x[55] + 2(x[10]^2)*x[2]*x[49]*x[55]*x[61] + 4x[19]*x[24]*x[30]*x[46]*x[47]*x[51]*x[58]
                                         x[11]*x[13]*x[25]*x[44] + 2x[2]*x[8]*x[10]*x[55] + 4x[19]*x[30]*x[34]*x[46]*x[51]*x[54]
                                         2(x[10]^2)*x[49] + x[10]*x[11]*x[13]*x[33]*x[44] + 4x[19]*x[30]*x[34]*x[47]*x[51]*x[54]
 x[6]*x[10]*x[11]*x[13]*x[25]*x[33]*x[44] + 2(x[10]^3)*x[8]*x[16]*x[22]*x[49]*x[61] + 4x[19]*x[24]*x[30]*x[46]*x[47]*x[51]*x[62]

In [102]:
# mons_rand4=rand(mons,4)
# polyn=0
# for i in 1:4
#     polyn=polyn+coeffs[i]*mons_rand4[i]
# end
# polyn

2x[13]*x[17]*x[39]*x[41] + 2x[1]*x[11]*x[20]*x[37]*x[40]*x[45]